In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Données

In [2]:
def loadMovieLens(path='../Data/ml-100k'):
    # Get movie titles | not very useful here
    movies={}
    for line in open(path+'/u.item'):
        (id,title)=line.split('|')[0:2]
        movies[id]=title

    # Load data
    prefs={} #dic {user :{item : rating}}
    times={} #dic {user :{item : timestamps}}
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
        times.setdefault(user,{})
        times[user][movies[movieid]]=float(ts)
    return prefs,times


In [16]:
def get_triple(data):
    '''returns triples user,item,value with dic {user :{item : value}}'''
    triple = []
    for u in data.keys():
        for i in data[u].keys():
            triple.append([u,i,data[u][i]])
    return triple

def get_dic_for_user(triples):
    ''' for Baseline'''
    dic_users={}
    for t in triples:
        if not t[0] in dic_users:
            dic_users[t[0]]={}
        dic_users[t[0]][t[1]]=float(t[2])
    return dic_users


def get_dic_for_item(triples):
    ''' for Baseline'''
    dic_users={}
    for t in triples:
        if not t[0] in dic_users:
            dic_users[t[1]]={}
        dic_users[t[1]][t[0]]=float(t[2])
    return dic_users

def getTimeBins(triples, timedic, nbins):
    ''' decoupage en nbins des timestamps'''
    timestamps = np.zeros(len(triples))
    for i,c in enumerate(triples):
        timestamps[i] = timedic[c[0]][c[1]]
    time_bins = np.linspace(np.min(timestamps), np.max(timestamps), nbins+1)
    times = np.zeros(len(triples))
    for i in xrange(1,len(time_bins)):
        times = times + (timestamps > time_bins[i])
    return times

In [4]:
def splitTrainTest(triples, split_pourcent) :
    perm = np.random.permutation(triples)
    Index = int(split_pourcent * len(triples))
    return perm[Index:], perm[:Index]

# Baseline

Dans le but d'évaluer notre modèle, nous comparerons nos modèles à une baseline. Deux modèles simples peuvent être envisagé: note moyenne par utilisateur et note moyenne par item. On attribue à notre ensemble test soit la moyenne de l'utilisateur ou la moyenne de l'item (ici le film).

In [63]:
class MeanUsers():
    def __init__(self):
        self.mean = {}
        
    def fit(self, userData):
        for user in userData.keys():
            #Initialisation de la moyenne
            self.mean[user] = 0
            for item in userData[user].keys():
                self.mean[user] = self.mean[user] + userData[user][item]
            self.mean[user] = self.mean[user] / len(userData[user])
            
    def predict(self, triple_test):
        pred = np.zeros(len(triple_test))
        for ind,c in enumerate(triple_test):
            pred[ind] = self.mean[c[0]]
        return pred
    
    def score(self,triples_test):
        pred = self.predict(triples_test)
        return ((pred - np.array(triples_test[:,2], float)) ** 2).mean()

class MeanItems():
    def __init__(self):            
        self.mean = {}
        
    def fit(self, itemData):
        for item in itemData.keys():
            #Initialisation de la moyenne
            self.mean[item] = 0
            for user in itemData[item].keys():
                self.mean[item] = self.mean[item] + itemData[item][user]
            self.mean[item] = self.mean[item] / len(itemData[item])
            
    def predict(self, triple_test):
        pred = np.zeros(len(triple_test))
        for ind,t in enumerate(triple_test):
            pred[ind] = self.mean[t[1]]
        return pred
    
    def score(self,triples_test):
        pred = self.predict(triples_test)
        return ((pred - np.array(triples_test[:,2], float)) ** 2).mean()

# Factorisation matricielle sans Biais

In [76]:
class Factorisation_Matricielle():
    '''Systeme de recommandation avec factorisation matricielle
    Classique L2 sans bias
    '''
    
    def __init__(self, k, eps=0.001, max_epoch=1000, lamb=0.5, alternate=False):
        '''constructeur
        k: Dimension espace latent
        eps : pas du gradient
        max_epoch: nombre maximum d iteration 
        lamb : coefficient de regularisation
        alternate : optimisation alternée ou non
        '''
        self.k = k
        self.lamb = lamb
        self.eps = eps
        self.max_epoch = max_epoch
        self.alternate = alternate

        
    def fit(self,triples):
        #self.p matrice des users
        #self.q matrice des items
        self.p = {}
        self.q = {}
        
        #Initialisation du Loss   
        self.loss = []
        
        #optimisation alterné ou non
        if self.alternate:
            optimize_p = True
            optimize_q = False
        else:
            optimize_p = True
            optimize_q = True
        
        #initialisation des matrices d'embeddings
        for j in range(len(triples)):
                u = triples[j][0]
                i = triples[j][1]
                #On verifie si l'item et le user existe
                if u not in self.p.keys():
                    self.p[u] = np.random.rand(1,self.k) 
                if i not in self.q.keys():
                    self.q[i] = np.random.rand(self.k,1)  
        
        #Descente de gradient
        for it in range(self.max_epoch):
            loss = 0
            for j in range(len(triples)):
                ind = np.random.randint(len(triples))
                u = triples[ind][0] #user
                i = triples[ind][1] #item 
                r = triples[ind][2] #rating

                tmp = float(r)  - np.asscalar(self.p[u].dot(self.q[i]))
                if optimize_p:
                    self.p[u] = self.p[u] - self.lamb * self.eps * self.p[u] + self.eps * 2 * tmp * self.q[i].T
                if optimize_q:
                    self.q[i] = self.q[i] - self.lamb * self.eps * self.q[i] + self.eps * 2 * tmp * self.p[u].T

                loss = loss + tmp*tmp
            #optimisation alterné
            if self.alternate:
                optimize_p,optimize_q = optimize_q,optimize_p
                
            #affichage du loss
            if ((it)%(self.max_epoch*0.1) == 0) :
                print "loss at iteration {} : {}".format(it,loss/len(triples))
                print ""
                
                
    def predict(self, triplet_test):
        pred = np.zeros(len(triplet_test))
        for ind,t in enumerate(triplet_test):
            pred[ind] = np.asscalar(self.p[t[0]].dot(self.q[t[1]]))
        return pred
    
    def score(self,triples_test):
        pred = self.predict(triples_test)
        return ((pred - np.array(triples_test[:,2], float)) ** 2).mean()


# Factorisation matricielle avec biais user et item

De façon intuitif, on peut généralement observer des biais sur les utilisateurs et sur les items. En effet, certains films sont beaucoup plus notés que d'autres (du a leurs popularité) et certains utilisateurs notent mieux en moyenne mieux que d'autres (optimistes contre pessimistes)

Pour prendre en compte ces biais, nous allons conidéré que chaque utilisateur a une note moyenne et chaque item a également une note moyenne.

In [77]:
class Factorisation_Matricielle_Biais_User_Item():
    '''Systeme de recommandation avec factorisation matricielle
    Classique L2 avec biais utilisateurs et items
    '''
    
    def __init__(self, k, eps=0.001, max_epoch=1000, lamb=0.5, alternate=False):
        '''constructeur
        k: Dimension espace latent
        eps : pas du gradient
        max_epoch: nombre maximum d iteration 
        lamb : coefficient de regularisation
        alternate : optimisation alternée ou non
        '''
        self.k = k
        self.lamb = lamb
        self.eps = eps
        self.max_epoch = max_epoch
        self.alternate = alternate

        
    def fit(self,triples):
        #self.p matrice des users
        #self.q matrice des items
        self.p = {}
        self.q = {}
        self.bu = {}
        self.bi = {}
        self.mu = np.random.random() * 2 - 1
        
        #Initialisation du Loss   
        self.loss = []
        
        #optimisation alterné ou non
        if self.alternate:
            optimize_p = True
            optimize_q = False
        else:
            optimize_p = True
            optimize_q = True
        
        #initialisation des matrices d'embeddings
        for j in range(len(triples)):
                u = triples[j][0]
                i = triples[j][1]
                #On verifie si l'item et le user existe
                if u not in self.p.keys():
                    self.p[u] = np.random.rand(1,self.k)
                    self.bu[u] = np.random.rand() * 2 - 1
                if i not in self.q.keys():
                    self.q[i] = np.random.rand(self.k,1) 
                    self.bi[i] = np.random.rand() * 2 - 1
        
        #Descente de gradient
        for it in range(self.max_epoch):
            loss = 0
            for j in range(len(triples)):
                ind = np.random.randint(len(triples))
                u = triples[ind][0] #user
                i = triples[ind][1] #item 
                r = triples[ind][2] #rating

                tmp = float(r)  - np.asscalar(self.p[u].dot(self.q[i])) - self.mu - self.bi[i] - self.bu[u]
                if optimize_p:
                    self.p[u] = self.p[u] - self.lamb * self.eps * self.p[u] + self.eps * 2 * tmp * self.q[i].T
                    self.bu[u] = (1 - self.lamb * self.eps) * self.bu[u] + self.eps * 2 * tmp
                if optimize_q:
                    self.q[i] = self.q[i] - self.lamb * self.eps * self.q[i] + self.eps * 2 * tmp * self.p[u].T
                    self.bi[i] = (1 - self.lamb * self.eps) * self.bi[i] + self.eps * 2 * tmp
                self.mu = (1 - self.lamb * self.eps) * self.mu + self.eps * 2 * tmp
                loss = loss + tmp*tmp
            #optimisation alterné
            if self.alternate:
                optimize_p,optimize_q = optimize_q,optimize_p
                
            #affichage du loss
            if ((it)%(self.max_epoch*0.1) == 0) :
                print "loss at iteration {} : {}".format(it,loss/len(triples))
                print ""
                
                
    def predict(self, triplet_test):
        pred = np.zeros(len(triplet_test))
        for ind,t in enumerate(triplet_test):
            pred[ind] = self.mu + self.bu[t[0]] + self.bi[t[1]] + np.asscalar(self.p[t[0]].dot(self.q[t[1]]))
        return pred
    
    def score(self,triples_test):
        pred = self.predict(triples_test)
        return ((pred - np.array(triples_test[:,2], float)) ** 2).mean()

# Factorisation matricielle avec biais user,item et temporel

On observe également un biais temporel sur les notes. En effet, dans une courbe présenté par M. Denoyer, nous observons que les utilisateurs ont tendances à plus noter selon certaines périodes de l'année (par exemple Noel).

In [112]:
class Factorisation_Matricielle_Biais_User_Item_Temporel():
    '''Systeme de recommandation avec factorisation matricielle
    Classique L2 avec biais utilisateurs et items et temporel
    '''
    
    def __init__(self, k=10, bins_times = 5, eps=0.001, max_epoch=1000, lamb=0.5, alternate=False):
        '''constructeur
        k: Dimension espace latent
        eps : pas du gradient
        max_epoch: nombre maximum d iteration 
        lamb : coefficient de regularisation
        alternate : optimisation alternée ou non
        '''
        self.k = k
        self.bins_times = bins_times
        self.lamb = lamb
        self.eps = eps
        self.max_epoch = max_epoch
        self.alternate = alternate

        
    def fit(self,triples,times):
        #self.p matrice des users
        #self.q matrice des items
        self.p = {}
        self.q = {}
        self.bu = {}
        self.bi = {}
        self.mu = np.random.random(self.bins_times) * 2 - 1
        
        #Initialisation du Loss   
        self.loss = []
        
        #optimisation alterné ou non
        if self.alternate:
            optimize_p = True
            optimize_q = False
        else:
            optimize_p = True
            optimize_q = True
        
        #initialisation des matrices d'embeddings
        for j in range(len(triples)):
                u = triples[j][0]
                i = triples[j][1]
                #On verifie si l'item et le user existe
                if u not in self.p.keys():
                    self.p[u] = np.random.rand(1,self.k)
                    self.bu[u] = np.random.rand(self.bins_times) * 2 - 1
                if i not in self.q.keys():
                    self.q[i] = np.random.rand(self.k,1)
                    self.bi[i] = np.random.rand(self.bins_times) * 2 - 1
        
        #Descente de gradient
        for it in range(50):
            loss = 0
            for j in range(len(triples)):
                ind = np.random.randint(len(triples))
                u = triples[ind][0] #user
                i = triples[ind][1] #item 
                r = triples[ind][2] #rating
                time = times[ind]

                tmp = float(r)  - np.asscalar(self.p[u].dot(self.q[i])) - self.mu[time] - self.bi[i][time] - self.bu[u][time]
                if optimize_p:
                    self.p[u] = self.p[u] - self.lamb * self.eps * self.p[u] + self.eps * 2 * tmp * self.q[i].T
                    self.bu[u][time] = (1 - self.lamb * self.eps) * self.bu[u][time] + self.eps * 2 * tmp
                if optimize_q:
                    self.q[i] = self.q[i] - self.lamb * self.eps * self.q[i] + self.eps * 2 * tmp * self.p[u].T
                    self.bi[i][time] = (1 - self.lamb * self.eps) * self.bi[i][time] + self.eps * 2 * tmp
                self.mu[time] = (1 - self.lamb * self.eps) * self.mu[time] + self.eps * 2 * tmp
                loss = loss + tmp*tmp
            self.loss.append(loss)
            #optimisation alterné
            if self.alternate:
                optimize_p,optimize_q = optimize_q,optimize_p
                
            #affichage du loss
            if ((it)%(self.max_epoch*0.1) == 0) :
                print "loss at iteration {} : {}".format(it,loss/len(triples))
                print ""
                
                
    def predict(self, triplet_test,times):
        pred = np.zeros(len(triplet_test))
        for ind,t in enumerate(triplet_test):
            pred[ind] = self.mu[times[ind]] + self.bu[t[0]][times[ind]] + self.bi[t[1]][times[ind]] + np.asscalar(self.p[t[0]].dot(self.q[t[1]]))
        return pred
    
    def score(self,triples_test):
        pred = self.predict(triples_test)
        return ((pred - np.array(triples_test[:,2], float)) ** 2).mean()

# Comparaison des modeles

## Preparation des données

In [113]:
data, times = loadMovieLens()
triples = get_triple(data)

triples_train, triples_test = splitTrainTest(triples , 0.1)

UserData_train = get_dic_for_user(triples_train)
ItemData_train = get_dic_for_item(triples_train)


In [118]:
def userInTrain(triples_train):
    user = []
    for element in triples_train:
        user.append(element[0])
    return set(user)

def itemInTrain(triples_train):
    item = []
    for element in triples_train:
        item.append(element[1])
    return set(item)

def delUserItemFromTest(triples_test,userTrain,itemTrain):
    toDel = []
    for i,t in enumerate(triples_test):
        if t[0] not in userTrain:
            toDel.append(i)
        elif t[1] not in itemTrain:
            toDel.append(i)
    return np.delete(triples_test, toDel, 0)


In [119]:
userTrain = userInTrain(triples_train)
itemTrain = itemInTrain(triples_train)

#supression des triples dont on ne connait ni le user ou litem
triples_test = delUserItemFromTest(triples_test,userTrain,itemTrain)

UserData_test = get_dic_for_user(triples_test)
ItemData_test = get_dic_for_item(triples_test)


In [120]:
print triples_test.shape
print triples_train.shape

(9956, 3)
(89724, 3)


In [121]:
bins_times = 5
trainTimes = getTimeBins(triples_train, times, bins_times)
testTimes = getTimeBins(triples_test, times, bins_times)


In [122]:
MeanUser = MeanUsers()
MeanUser.fit(UserData_train)
print 'test erreur moyenne user: {}'.format(MeanUser.score(triples_test))

test erreur moyenne user: 1.08640355831


In [123]:
MeanItem = MeanItems()
MeanItem.fit(ItemData_train)
print 'test erreur moyenne item: {}'.format(MeanItem.score(triples_test))

test erreur moyenne item: 2.18119726798


In [86]:
k = 5
eps = 0.001
max_epoch = 50
lamb = 0.2
SR = Factorisation_Matricielle(k,eps,max_epoch,lamb)
SR.fit(triples_train)

loss at iteration 0 : 3.74304610439

loss at iteration 5 : 0.968604039549

loss at iteration 10 : 0.895185149817

loss at iteration 15 : 0.867189257012

loss at iteration 20 : 0.856250716388

loss at iteration 25 : 0.846256291322

loss at iteration 30 : 0.845500139891

loss at iteration 35 : 0.839855166803

loss at iteration 40 : 0.83257425937

loss at iteration 45 : 0.831192840806



In [88]:
print 'test erreur sans biais: {}'.format(SR.score(triples_test))

test erreur sans biais: 0.887710342468


In [90]:
SR_with_bias = Factorisation_Matricielle_Biais_User_Item(k,eps,max_epoch,lamb)
SR_with_bias.fit(triples_train)

loss at iteration 0 : 1.65900375771

loss at iteration 5 : 0.94440218995

loss at iteration 10 : 0.881711567467

loss at iteration 15 : 0.865785005575

loss at iteration 20 : 0.85178152145

loss at iteration 25 : 0.847941690379

loss at iteration 30 : 0.842054102166

loss at iteration 35 : 0.837511524624

loss at iteration 40 : 0.826877275397

loss at iteration 45 : 0.826918907865



In [92]:
print 'test erreur sans biais: {}'.format(SR_with_bias.score(triples_test))

test erreur sans biais: 0.886626674726


In [124]:
SR_with_bias_temp = Factorisation_Matricielle_Biais_User_Item_Temporel(k,eps,max_epoch,lamb)
SR_with_bias_temp.fit(triples_train,trainTimes)

/home/gozuslayer/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:29: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/gozuslayer/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:49: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/gozuslayer/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:52: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/gozuslayer/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:64: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


IndexError: index 3 is out of bounds for axis 0 with size 0